In [8]:
import numpy as np

In [14]:
# 1 CHOSE STATE VARIABLES - assume here you are tracking a drones x and y position and velocity

x = 0
y = 0
x_dot = 0
y_dot = 0

state_x = np.array(([x, y, x_dot, y_dot]))
state_x


array([0, 0, 0, 0])

In [16]:
# 2 DESIGN STATE TRANSITION FUNCTION (F) RECALL THIS TURNS TO (A) WHEN DEALING WITH CONTINUOUS
# recall that each variable in the state vector state_x needs an innovation equation 
# x = 1x + delta_t*x_dot + 0y + 0y_dot
# x_dot = x_dot
# y = 0x + 0x_dot + 1y + delta_t * y_dot
# y_dot = y_dot
# thus

dt = 1. # time step 1 second
F = np.array([[1, dt, 0,  0],
              [0,  1, 0,  0],
              [0,  0, 1, dt],
              [0,  0, 0, 1]])

In [22]:
# 3 DEISNG THE PROCESS NOISE MATRIX (Q) here we use constant velocity model

def Q_discrete_white_noise(dim, dt=1., var=1., block_size=1, order_by_dim=True):
    """
    Returns the Q matrix for the Discrete Constant White Noise
    Model. dim may be either 2, 3, or 4 dt is the time step, and sigma
    is the variance in the noise.
    Q is computed as the G * G^T * variance, where G is the process noise per
    time step. In other words, G = [[.5dt^2][dt]]^T for the constant velocity
    model.
    Parameters
    -----------
    dim : int (2, 3, or 4)
        dimension for Q, where the final dimension is (dim x dim)
    dt : float, default=1.0
        time step in whatever units your filter is using for time. i.e. the
        amount of time between innovations
    var : float, default=1.0
        variance in the noise
    block_size : int >= 1
        If your state variable contains more than one dimension, such as
        a 3d constant velocity model [x x' y y' z z']^T, then Q must be
        a block diagonal matrix.
    order_by_dim : bool, default=True
        Defines ordering of variables in the state vector. `True` orders
        by keeping all derivatives of each dimensions)
        [x x' x'' y y' y'']
        whereas `False` interleaves the dimensions
        [x y z x' y' z' x'' y'' z'']
    Examples
    --------
    >>> # constant velocity model in a 3D world with a 10 Hz update rate
    >>> Q_discrete_white_noise(2, dt=0.1, var=1., block_size=3)
    array([[0.000025, 0.0005  , 0.      , 0.      , 0.      , 0.      ],
           [0.0005  , 0.01    , 0.      , 0.      , 0.      , 0.      ],
           [0.      , 0.      , 0.000025, 0.0005  , 0.      , 0.      ],
           [0.      , 0.      , 0.0005  , 0.01    , 0.      , 0.      ],
           [0.      , 0.      , 0.      , 0.      , 0.000025, 0.0005  ],
           [0.      , 0.      , 0.      , 0.      , 0.0005  , 0.01    ]])
    References
    ----------
    Bar-Shalom. "Estimation with Applications To Tracking and Navigation".
    John Wiley & Sons, 2001. Page 274.
    """

    if dim not in [2, 3, 4]:
        raise ValueError("dim must be between 2 and 4")

    if dim == 2:
        Q = [[.25*dt**4, .5*dt**3],
             [ .5*dt**3,    dt**2]]
    elif dim == 3:
        Q = [[.25*dt**4, .5*dt**3, .5*dt**2],
             [ .5*dt**3,    dt**2,       dt],
             [ .5*dt**2,       dt,        1]]
    else:
        Q = [[(dt**6)/36, (dt**5)/12, (dt**4)/6, (dt**3)/6],
             [(dt**5)/12, (dt**4)/4,  (dt**3)/2, (dt**2)/2],
             [(dt**4)/6,  (dt**3)/2,   dt**2,     dt],
             [(dt**3)/6,  (dt**2)/2 ,  dt,        1.]]

    if order_by_dim:
        return block_diag(*[Q]*block_size) * var
    return order_by_derivative(array(Q), dim, block_size) * var
R_std = 0.35
Q_std = 0.044

q = Q_discrete_white_noise(dim=4, dt=dt, )

In [4]:
# 4 DESIGN THE CONTROL FUNCTION (B) for now zero

In [5]:
# 5 DESIGN THE MEASUREMENT FUNCTION (H)
# recall that te measurement fucntion H defines how we go from the state vriable to the measurements using the eqaution z = Hx
# ehere we have measurements only for x and y soz will be designed as [x y].T 
# we have a state varialbe size 4x1 
# if z = Hx and z is 2x1 -> [x y].T we can deduce that H ix 2x4 since (2x1) = (2x4)(4x1)

H = np.array([[ 1/0.3048, 0, 0 ,0],
              [0 , 0, 1/0.3048, ]])

In [20]:
# 6 DESIGN THE MEASUREMENT NOISE MATRIX (R)
# assume that x and y variealbe have independent with gaussian processes. Say variance for x and y is 5 meetsrs
R = np.array([[5, 0],
              [0, 5]])

In [21]:
# THE COVARIANCE MATRIX P 
p = np.eye(4)*500